In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/umasreeram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/umasreeram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


__Creating functions to clean tweets__

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
#import lang_detect
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from langdetect import detect



def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words(words)
    return " ".join(stems)
    lemmas = lemmatize_verbs(stems)
    return " ".join(lemmas)

def only_english(mystr):
    if lang_detect(mystr)=='en':   #give up
        return mystr
    else :
        return ''


def clean_string(mystr):
    mystr=mystr.lower()
    mystr=re.sub(r"\\\w+", " ", mystr)
    mystr=re.sub(r"\@\w+"," ",mystr)
    mystr=re.sub(r"\#\w+"," ",mystr)
    mystr=re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"," ",mystr)
    
    mystr=mystr[2:-1]
    
    cleantext = "".join([x.lower() if (x.isalpha() or x.isspace()) else ' ' for x in mystr])
    return cleantext

def remove_stop_words(mystr):
    word_list= mystr.split()
    cleaned_word_list=[w for w in word_list if w not in stop_words and len(w)>=2]
    
    return cleaned_word_list
   
    
def join_sep(mylist):
    return " ".join(mylist)
   


In [4]:
# impcolumns=list(vertical_stack.columns[np.argsort(chi2score[0])[::-1]][:1000])  #from a previous run, now i have cut those cells
# #print(impcolumns)

# with open("impcolumns.txt", 'w') as f:
#     for s in impcolumns:
#         f.write(s + '\n')

#read these columns from a text file calledimpcolumns.txt if needed

with open('impcolumns.txt', 'r') as f:
    impcolumns = f.readlines()
    

    
impcolumns = [line[:-1] for line in impcolumns]
#print(impcolumns)
vertical_stack=impcolumns

__Loading data and packages__

In [5]:
import json
with open('approach2_final_data.json', 'r') as fp:
    tweet_data = json.load(fp)

In [6]:
#Using oversampled data from drive

In [7]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

__Loading Classifier and prediction__

In [8]:
import pickle
from sklearn.externals import joblib

/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [9]:
Pkl_Filename='Classifiers/SVM_bagging.pkl'
with open(Pkl_Filename, 'rb') as file:  
    classifier = clf = joblib.load(file)


classifier

/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator BaggingClassifier from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


BaggingClassifier(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                           fit_intercept=True,
                                           intercept_scaling=1,
                                           loss='squared_hinge', max_iter=1000,
                                           multi_class='ovr', penalty='l2',
                                           random_state=0, tol=1e-05,
                                           verbose=0),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=50, n_jobs=None,
                  oob_score=False, random_state=0, verbose=0, warm_start=False)

__Extras__

In [10]:
# X_new.to_csv('X_train_oversampled.csv', index = False, header=True)
# y_new.to_csv('y_train_oversampled.csv', index = False, header=True)
# X_test.to_csv('X_test.csv', index = False, header=True)
# y_test.to_csv('y_test.csv', index = False, header=True)

### Making tfidf from tweets and score


In [17]:


def return_score(user_tweets):     #here user_tweets would be a dict with key as the username and value as the tweets 

     
    dataframes = {}
    dataframes_empty = {}


    for x in user_tweets:
    #print(x)
        file=pd.DataFrame(user_tweets[x],columns=['id','text'])
    
        file['clean_text']=file['text'].apply(clean_string)
        file['word_list']=file['clean_text'].apply(remove_stop_words)
        file['word_list']=file['word_list'].apply(join_sep)
        #file['word_list']=file['word_list'].apply(only_english)
    
        file=file.dropna()
        file=file.loc[file['word_list']!='']
    
    
        try:
            vectorizer = TfidfVectorizer()
        
            tfidfmatrix = vectorizer.fit_transform(file['word_list'])
            vocab = vectorizer.get_feature_names()

            tfidf_data=tfidfmatrix.toarray()
        
            tfidf_pd=pd.DataFrame(data=tfidf_data,columns=vocab,index=file['id'])
        
            tfidf_pd = tfidf_pd.drop(tfidf_pd.columns.difference(impcolumns),1)
       
        
            dataframes[x] = tfidf_pd
            #print("Success",x)
    
        except ValueError as e: #happens when the vocabulary is empty
        
            #print("Failure",x,file['text'])
            dataframes_empty[x]='N/A'
        
        
            continue
            
    tesla_followers=dataframes
    feature_list=impcolumns
    #UNIFORM PADDING
    for key in tesla_followers:
        test=tesla_followers[key]
    
        #print(len(test.columns))
    

        toadd=list(set(feature_list)-set(test.columns))


        toadd_df=pd.DataFrame(np.zeros((test.shape[0], len(toadd))),columns=toadd)
        test.reset_index(drop=True, inplace=True)
        toadd_df.reset_index(drop=True, inplace=True)


        test=pd.concat([test,toadd_df],axis=1)
    
        tesla_followers[key]=test
        #print(test.shape)

    
    #SCORE    
    tesla_tweet_prediction=[]
    score={}
    for key in tesla_followers:
        test=tesla_followers[key]
        current_user_pred=classifier.predict(test)
        tesla_tweet_prediction.append(current_user_pred)
        score[key]=sum(current_user_pred)/len(current_user_pred)   
    

    import collections
    from collections import OrderedDict
    from operator import itemgetter 
    sorted_score = OrderedDict(sorted(score.items(), key=itemgetter(1),reverse=True))
    sorted_score.update(dataframes_empty)    #adding users with No tweets
    
    return sorted_score




In [45]:
first_level={}

for k, v in tweet_data['tesla'].items():
    #print(k)
    #print(v.keys())
    first_level[k]=v['tweets']
    

In [48]:
print(return_score(first_level))

OrderedDict([('mahamunidurai', 0.5294117647058824), ('iamdarkwing', 0.5), ('KedriW', 0.5), ('Alex62249401', 0.48148148148148145), ('Angelin57066746', 0.4675324675324675), ('Opsilon_Y', 0.4444444444444444), ('BRITTNEY0623', 0.4), ('bait4love', 0.3230769230769231), ('qnguyvn', 0.30434782608695654), ('makoma_pulcia', 0.26666666666666666), ('Prabhas27919923', 0.2631578947368421), ('Nickie52611168', 0.2222222222222222), ('UnicornModest', 0.2), ('ercanyildirimtr', 0.1377551020408163), ('nook2ladiez', 0.0), ('tiberiu99783268', 0.0), ('MarkWel72212278', 0.0), ('AnnaSamkov1', 0.0), ('abdoabeed17', 0.0), ('JojoWest8', 0.0), ('Allan_Cortez_V', 0.0), ('koukoucang', 0.0), ('Thierno79594590', 'N/A'), ('Rooney05091', 'N/A'), ('Rahim42420139', 'N/A'), ('Sarvino46081636', 'N/A'), ('MiikaTolonen5', 'N/A'), ('BEATA18510687', 'N/A'), ('LukasPidgeon', 'N/A'), ('vicman7721', 'N/A'), ('Ajaycha44323532', 'N/A'), ('ANXDRXN1', 'N/A'), ('AbuHuzain', 'N/A'), ('ZtofKr', 'N/A'), ('Niveen50330395', 'N/A'), ('Gerardo

In [52]:
second_level={}
for k, v in tweet_data['tesla'].items():
    second_level[k] = {}
    follwer_list = []
    user_tweets={}
    if isinstance(v, dict):
        for i, j in v.items():
            
            if not (i == "tweets" or i == 'location'):
                    follwer_list.append(i)
                    if j['tweets']!="error":
                        user_tweets[i] = j['tweets']
    second_level[k] = return_score(user_tweets)
                        

                            

In [53]:

import json
with open('hierarchy_scores.json', 'w') as fp:
    json.dump(second_level, fp)

In [79]:
#KMEANS
from sklearn.cluster import KMeans

def kmeans_avg(scores_list):
    X = np.array(scores_list)
    if len(scores_list)>5:
        kmeans = KMeans(n_clusters=3, random_state=0).fit(X.reshape(-1, 1))
        label = kmeans.labels_
        counts = np.bincount(label)
        cluster_no = (np.argmax(counts))
        result = np.array(list(zip(X,label)))
        res = result[result[:,1] == cluster_no]
        web_score = np.mean(res[:,0])
    else:
        #print(scores_list)
        web_score=np.mean(scores_list)
    
    return web_score



In [81]:
second_level_list={}
for x,y in second_level.items():
    try:
        if not(y.values()==[] or y.values()==['N/A']):
            second_level_list[x]=kmeans_avg(list(y.values()))
        else:
            second_level_list[x]='N/A(web)'
    except:
        second_level_list[x]='N/A(web)'
    
    
    
print(len(second_level_list))

49


/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean o

In [63]:
ctr=0
for k,v in tweet_data['tesla'].items():
    #print(k)
    
    if isinstance(v, dict):
        for i, j in v.items():
            if not (i == "tweets" or i == 'location'):
                #print(i)
                ctr+=1
                
                
print(ctr)                

228


In [30]:
# a_file = open("sorted_score_first_level.csv", "w")
# #a_dict = {"a": 1, "b": 2}
# import csv

# writer = csv.writer(a_file)
# for key, value in sorted_score.items():
#     writer.writerow([key, value])

# a_file.close()